In [508]:
import numpy.linalg as linalg
import numpy as np
import pandas as pd
import plotly.express as px

from random import choice

In [509]:
epsilon = 0.0001

### Метод Гаусса

In [510]:
def solve_Gauss(A, b, eps):
    n = A.shape[0]
    A = np.hstack((A, b))
    x = np.zeros(n)

    for k in range(n):
        A[k, :] /= A[k, k]
        for i in range(k + 1, n):
            A[i, :] -= A[k, :] * A[i, k]

    for i in range(0, n)[::-1]:
        x[i] = A[i, n] - np.dot(A[i, :-1], x)
    return x

### Метод Гаусса с выбором главного элемента

In [511]:
def transform(A, b):
    n, m = A.shape
    A = np.hstack((A, b))

    c = np.array(range(A.shape[1])).reshape((1, A.shape[1]))
    A = np.vstack((A, c))

    for k in range(n):
        for p in range(k, n):
            for q in range(k, m):
                if abs(A[p, q]) > abs(A[k, k]):
                    A[[k, p], :] = A[[p, k], :]
                    A[:, [k, q]] =  A[:, [q, k]]

    b = A[:-1, -1].reshape((n, 1))
    c = A[-1, :-1]
    A = A[:-1, :-1]

    return A, b, c

In [512]:
def transform_solver(A, b, eps):
    n = A.shape[0]
    b = b.reshape(n, 1)
    A, b, c = transform(A, b)
    
    x = np.zeros((n, 1))
    unordered_x = solve_Gauss(A.copy(), b.copy(), eps)

    for i in range(n):
        x[int(c[i])] = unordered_x[i]
    x = np.array(x)
    return x.ravel()

### LU-разложение

In [513]:
def LU(A):
    n = A.shape[0]
    L = np.zeros(A.shape)
    U = np.zeros(A.shape)

    # LU-разложение
    for i in range(n):
        for j in range(i, n):
            L[j, i] = A[j, i] - np.dot(L[j, :i], U[:i, i])
            U[i, j] = (A[i, j] - np.dot(L[i, :i], U[:i, j])) / L[i, i]

    return L, U

In [514]:
def solve_LU(A, b, eps):
    b = b.reshape(A.shape[0], 1)
    L, U = LU(A.copy())
    y = solve_Gauss(L, b.copy(), eps)
    y = y.reshape(y.shape[0], 1)
    x = solve_Gauss(U, y, eps)
    return x


### Итерационный метод Гаусса-Зейделя


In [515]:
def seidel(A, b, eps):
    L_inv = linalg.inv(np.tril(A))
    U = np.triu(A, 1)
    x0 = np.zeros(b.shape[0])

    x = x0
    while True:
        x_new = L_inv @ (b - U @ x)
        if np.allclose(x, x_new, eps):
            break
        x = x_new
    return x


### Генератор матриц с диагональным преобладанием


In [516]:
def gen_matrix(n, k):
    a = np.zeros((n, n))
    pos_ij = [0, -1, -2, -3, -4]
    for i in range(n):
        for j in range(n):
            if i == j: continue
            a[i, j] = choice(pos_ij)
        a[i, i] = -np.sum(a[i]) if i > 0 else -np.sum(a[i]) + 10**(-k)
    return a


### Генератор матриц Гильберта


In [517]:
def gen_hilbert(n):
    a = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            a[i, j] = 1 / (i + j + 1)
    return a


### Тест


In [518]:
A = np.array(np.mat('10 3 -1; 2 5 2; 0 3 -4')).astype(float)
b = np.array([-22, -33, -47]).T.astype(float)
x_true = [1, -9, 5]

methods = [(seidel, 'Seidel method'), (transform_solver, 'Gauss method'), (solve_LU, 'LU decomposition')]

for method in methods:
    %time x = method[0](A.copy(), b.copy(), epsilon)
    print(f'{method[1]}: {x}, relative error: {linalg.norm(x - x_true) / linalg.norm(x_true)}\n')

CPU times: total: 0 ns
Wall time: 1 ms
Seidel method: [ 0.9999367  -9.00005908  4.99995569], relative error: 9.40239416995768e-06

CPU times: total: 0 ns
Wall time: 0 ns
Gauss method: [ 1. -9.  5.], relative error: 4.2931724355351243e-17

CPU times: total: 0 ns
Wall time: 0 ns
LU decomposition: [ 1. -9.  5.], relative error: 4.2931724355351243e-17




### Исследование методов на матрицах с задаваемым диагональным преобладанием


In [519]:
testing_df1 = pd.DataFrame({'method': [], 'k': [], 'condition num': [], 'rel error': []})
methods = [(seidel, 'Seidel method'), (transform_solver, 'Gauss method'), (solve_LU, 'LU decomposition')]

for k in np.linspace(1e-10, 2, 30):
    A_k = gen_matrix(10, k)
    F_k = A_k @ np.arange(1, A_k.shape[0] + 1).T

    for method in methods:
        try:
            x = method[0](A_k.copy(), F_k.copy(), epsilon)
            testing_df1.loc[len(testing_df1.index)] = [method[1], k, linalg.cond(A_k), linalg.norm(x - np.arange(1, A_k.shape[0] + 1)) / linalg.norm(np.arange(1, A_k.shape[0] + 1))]
        except linalg.LinAlgError:
            print('Singular!')

In [520]:
fig = px.line(testing_df1, x='k', y='condition num', markers='o', title='cond(k)', height=500, width=1000)
fig.update_layout(margin=dict(l=75, r=20, t=50, b=15))

In [521]:
fig = px.line(testing_df1, x='k', y='rel error', color='method', markers='o', title='err(k)', height=500, width=1000)
fig.update_layout(legend=dict(x=0.005, y=0.99),
                  margin=dict(l=75, r=20, t=50, b=15))


### Исследование методов на матрицах Гильберта


In [522]:
testing_df2 = pd.DataFrame({'method': [], 'size': [], 'condition num': [], 'rel error': []})
methods = [(seidel, 'Seidel method'), (transform_solver, 'Gauss method'), (solve_LU, 'LU decomposition')]

for size in np.arange(2, 203, 20):
    A_k = gen_hilbert(size)
    F_k = A_k @ np.arange(1, A_k.shape[0] + 1).T

    for method in methods:
        try:
            x = method[0](A_k.copy(), F_k.copy(), epsilon)
            testing_df2.loc[len(testing_df2.index)] = [method[1], size, linalg.cond(A_k), linalg.norm(x - np.arange(1, A_k.shape[0] + 1)) / linalg.norm(np.arange(1, A_k.shape[0] + 1))]
        except linalg.LinAlgError:
            print('Singular!')

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\666460282.py:10: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\666460282.py:10: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\495086872.py:7: RuntimeWarning:

divide by zero encountered in divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\495086872.py:7: RuntimeWarning:

invalid value encountered in divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\666460282.py:10: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\666460282.py:10: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\495086872.py:7: RuntimeWarning:

divide by zero encountered in divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\495086872.py:7: RuntimeWarning:

invalid value encountered in divide

C:\U

In [523]:
fig = px.line(testing_df2, x='size', y='condition num', markers='o', title='cond(n)', height=500, width=1000)
fig.update_layout(margin=dict(l=75, r=20, t=50, b=15))

In [524]:
fig = px.line(testing_df2, x='size', y='rel error', color='method', markers='o', title='err(n)', height=500, width=1000)
fig.update_layout(legend=dict(x=0.005, y=0.99),
                  margin=dict(l=75, r=20, t=50, b=15))


### Сравнение методов


In [525]:
for size in [2, 10, 50, 100, 200]:
    A_k = gen_matrix(size, 0.01)
    F_k = A_k @ np.arange(1, A_k.shape[0] + 1).T

    for method in methods:
        try:
            %time x = method[0](A_k.copy(), F_k.copy(), epsilon)
            print(f'{method[1]}, n = {size}, relative residual: {linalg.norm(x - np.arange(1, A_k.shape[0] + 1)) / linalg.norm(np.arange(1, A_k.shape[0] + 1))}\n')
        except linalg.LinAlgError:
            print('Singular!')

CPU times: total: 15.6 ms
Wall time: 2.1 ms
Seidel method, n = 2, relative residual: 0.0003124515859353202

CPU times: total: 0 ns
Wall time: 0 ns
Gauss method, n = 2, relative residual: 0.0

CPU times: total: 0 ns
Wall time: 0 ns
LU decomposition, n = 2, relative residual: 0.0

CPU times: total: 15.6 ms
Wall time: 23.8 ms
Seidel method, n = 10, relative residual: 0.0014357336173672203

CPU times: total: 0 ns
Wall time: 2.19 ms
Gauss method, n = 10, relative residual: 4.468260552481954e-15

CPU times: total: 0 ns
Wall time: 999 µs
LU decomposition, n = 10, relative residual: 6.378016783845158e-15

CPU times: total: 734 ms
Wall time: 541 ms
Seidel method, n = 50, relative residual: 0.00661162474299885

CPU times: total: 15.6 ms
Wall time: 22 ms
Gauss method, n = 50, relative residual: 3.55879823041959e-13

CPU times: total: 0 ns
Wall time: 10 ms
LU decomposition, n = 50, relative residual: 1.5983898287950513e-14

CPU times: total: 2.11 s
Wall time: 2.26 s
Seidel method, n = 100, relativ

In [526]:
for size in [2, 10, 50, 100, 200]:
    A_k = gen_hilbert(size)
    F_k = A_k @ np.arange(1, A_k.shape[0] + 1).T

    for method in methods:
        try:
            %time x = method[0](A_k.copy(), F_k.copy(), epsilon)
            print(f'{method[1]}, n = {size}, relative residual: {linalg.norm(x - np.arange(1, A_k.shape[0] + 1)) / linalg.norm(np.arange(1, A_k.shape[0] + 1))}\n')
        except linalg.LinAlgError:
            print('Singular!')

CPU times: total: 0 ns
Wall time: 2 ms
Seidel method, n = 2, relative residual: 0.0002559599715883447

CPU times: total: 0 ns
Wall time: 0 ns
Gauss method, n = 2, relative residual: 6.661338147750939e-16

CPU times: total: 0 ns
Wall time: 0 ns
LU decomposition, n = 2, relative residual: 6.661338147750939e-16

CPU times: total: 0 ns
Wall time: 114 ms
Seidel method, n = 10, relative residual: 0.0682840601913047

CPU times: total: 0 ns
Wall time: 987 µs
Gauss method, n = 10, relative residual: 0.00021865996799152874

CPU times: total: 0 ns
Wall time: 1e+03 µs
LU decomposition, n = 10, relative residual: 0.0002600663667431266

CPU times: total: 1.22 s
Wall time: 1.39 s
Seidel method, n = 50, relative residual: 0.037299798955959786

CPU times: total: 0 ns
Wall time: 20.5 ms
Gauss method, n = 50, relative residual: 42.623087538677595

CPU times: total: 15.6 ms
Wall time: 11.5 ms
LU decomposition, n = 50, relative residual: 73.80436767453013

CPU times: total: 1.44 s
Wall time: 1.45 s
Seidel 

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\666460282.py:10: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\666460282.py:10: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\495086872.py:7: RuntimeWarning:

divide by zero encountered in divide

C:\Users\moroz\AppData\Local\Temp\ipykernel_31848\495086872.py:7: RuntimeWarning:

invalid value encountered in divide

